# Stock Data & Sentiment Features Merger

Merge stock market data with sentiment analysis features for Nifty top 10 stocks.

In [23]:
import pandas as pd
import os
from pathlib import Path

In [24]:
nifty_top_tickers = [
    "RELIANCE",
    "HDFCBANK",
    "TCS",
    "BHARTIARTL",
    "ICICIBANK",
    "SBIN",
    "INFY",
    "KOTAKBANK",
    "BAJFINANCE",
    "HINDUNILVR"
]

In [25]:
# Define paths
stock_data_dir = 'stock_data_daily_processed'
sentiment_features_dir = 'final_sentiment_features'
output_dir = 'merged_stock_sentiment_data'

# Create output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

In [26]:
def merge_stock_sentiment_data(ticker):
    """
    Merge stock data with sentiment features for a given ticker.
    Ensures proper date alignment and correct column names.
    """
    # Load stock data - row 0 has feature names, skip rows 1-2 (Ticker and Date labels)
    stock_file = f'{stock_data_dir}/{ticker}_daily_data.csv'
    stock_df = pd.read_csv(stock_file, header=0, skiprows=[1, 2])
    
    # Rename first column to 'Date'
    stock_df.rename(columns={stock_df.columns[0]: 'Date'}, inplace=True)
    
    # Load sentiment features
    sentiment_file = f'{sentiment_features_dir}/{ticker}_features.csv'
    sentiment_df = pd.read_csv(sentiment_file)
    
    # Convert date columns to datetime
    stock_df['Date'] = pd.to_datetime(stock_df['Date'])
    sentiment_df['date'] = pd.to_datetime(sentiment_df['date'])
    
    # Merge on date
    merged_df = pd.merge(
        stock_df, 
        sentiment_df, 
        left_on='Date', 
        right_on='date', 
        how='left'
    )
    
    # Drop the duplicate date column
    merged_df = merged_df.drop('date', axis=1)
    
    # Fill missing sentiment values with 0
    sentiment_cols = ['avg_polarity', 'avg_confidence', 'news_article_count', 
                      'sentiment_change', 'momentum_3d', 'momentum_5d']
    merged_df[sentiment_cols] = merged_df[sentiment_cols].fillna(0)
    
    # Convert numeric columns to appropriate types
    for col in merged_df.columns:
        if col != 'Date':
            merged_df[col] = pd.to_numeric(merged_df[col], errors='coerce')
    
    return merged_df

In [27]:
# Process all tickers and save merged data
for ticker in nifty_top_tickers:
    print(f"Processing {ticker}...")
    
    # Merge the data
    merged_df = merge_stock_sentiment_data(ticker)
    
    # Save to CSV
    output_file = f'{output_dir}/{ticker}_merged_data.csv'
    merged_df.to_csv(output_file, index=False)
    print(f"  ✓ Saved: {output_file} - Shape: {merged_df.shape}\n")

print("✓ All files merged and saved successfully!")

Processing RELIANCE...
  ✓ Saved: merged_stock_sentiment_data/RELIANCE_merged_data.csv - Shape: (1461, 48)

Processing HDFCBANK...
  ✓ Saved: merged_stock_sentiment_data/HDFCBANK_merged_data.csv - Shape: (1461, 48)

Processing TCS...
  ✓ Saved: merged_stock_sentiment_data/TCS_merged_data.csv - Shape: (1461, 48)

Processing BHARTIARTL...
  ✓ Saved: merged_stock_sentiment_data/HDFCBANK_merged_data.csv - Shape: (1461, 48)

Processing TCS...
  ✓ Saved: merged_stock_sentiment_data/TCS_merged_data.csv - Shape: (1461, 48)

Processing BHARTIARTL...
  ✓ Saved: merged_stock_sentiment_data/BHARTIARTL_merged_data.csv - Shape: (1461, 48)

Processing ICICIBANK...
  ✓ Saved: merged_stock_sentiment_data/ICICIBANK_merged_data.csv - Shape: (1461, 48)

Processing SBIN...
  ✓ Saved: merged_stock_sentiment_data/BHARTIARTL_merged_data.csv - Shape: (1461, 48)

Processing ICICIBANK...
  ✓ Saved: merged_stock_sentiment_data/ICICIBANK_merged_data.csv - Shape: (1461, 48)

Processing SBIN...
  ✓ Saved: merged_sto

In [28]:
# Verify the merged data quality
sample = pd.read_csv(f'{output_dir}/BAJFINANCE_merged_data.csv')

print("="*80)
print("MERGED DATA VERIFICATION")
print("="*80)
print(f"\nShape: {sample.shape}")
print(f"Date range: {sample['Date'].min()} to {sample['Date'].max()}")

print(f"\n{'-'*80}")
print("COLUMN STRUCTURE")
print(f"{'-'*80}")
print(f"\nTotal columns: {len(sample.columns)}")
print(f"\nStock features (41): {sample.columns[1:42].tolist()}")
print(f"\nSentiment features (6): {sample.columns[42:].tolist()}")

print(f"\n{'-'*80}")
print("DATA SAMPLE - First 5 rows with key columns")
print(f"{'-'*80}")
display_cols = ['Date', 'Close', 'Volume', 'log_return', 'target', 
                'avg_polarity', 'avg_confidence', 'news_article_count']
print(sample[display_cols].head())

print(f"\n{'-'*80}")
print("SENTIMENT DATA ALIGNMENT CHECK")
print(f"{'-'*80}")
print("\nRows with news articles:")
news_rows = sample[sample['news_article_count'] > 0][display_cols].head(10)
print(news_rows)

print(f"\n{'-'*80}")
print("MISSING VALUES CHECK")
print(f"{'-'*80}")
missing = sample.isnull().sum()
if missing.sum() > 0:
    print("\nColumns with missing values:")
    print(missing[missing > 0])
else:
    print("\n✓ No missing values found!")

print(f"\n{'-'*80}")
print("SENTIMENT FEATURES STATISTICS")
print(f"{'-'*80}")
sentiment_cols = ['avg_polarity', 'avg_confidence', 'news_article_count', 
                  'sentiment_change', 'momentum_3d', 'momentum_5d']
print(sample[sentiment_cols].describe())

print(f"\n{'='*80}")
print("✓ MERGE VERIFICATION COMPLETE")
print(f"{'='*80}")

MERGED DATA VERIFICATION

Shape: (1461, 48)
Date range: 2018-01-30 to 2023-12-29

--------------------------------------------------------------------------------
COLUMN STRUCTURE
--------------------------------------------------------------------------------

Total columns: 48

Stock features (41): ['Close', 'High', 'Low', 'Open', 'Volume', 'log_return', 'return_lag_1', 'return_lag_2', 'return_lag_3', 'return_lag_5', 'price_change_5d', 'price_change_10d', 'price_change_20d', 'high_low_ratio', 'open_close_ratio', 'volatility_5d', 'volatility_10d', 'volatility_20d', 'volume_change', 'volume_ma_5', 'volume_ma_20', 'volume_ratio_5d', 'volume_ratio_20d', 'price_volume_trend', 'sma_5', 'sma_10', 'sma_20', 'ema_10', 'ema_20', 'price_to_sma_5', 'price_to_sma_20', 'rsi_14', 'macd', 'macd_signal', 'macd_hist', 'bollinger_upper', 'bollinger_lower', 'bollinger_mid', 'bollinger_width', 'bollinger_position', 'target']

Sentiment features (6): ['avg_polarity', 'avg_confidence', 'news_article_count'